In [65]:
import re
import pandas as pd
from typing import List, Union
from fuzzywuzzy import fuzz
import ast

# Load your dataset
df = pd.read_csv('Spar_test.csv')
print(len(df))
relations = df['Q_type'].unique()
# Get the unique relations and their counts
relation_counts = df['Q_type'].value_counts()
# Create a dictionary of relations
relation_dict = relation_counts.to_dict()
# Print the relation dictionary
print("Q_type Dictionary",relation_dict)

import pandas as pd
import ast

# Load the dataset
df = pd.read_csv('Spar.csv')


# Convert string representations of lists to actual lists
df['Answer'] = df['Answer'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Initialize an empty list to store the processed dataframes
processed_dfs = []
n_samples = 35

# Iterate over each unique question type
for question in df['Q_type'].unique():
    # Filter the dataset for the current question type and length of Answer less than 2
    filtered_df = df[(df['Q_type'] == question) & (df['Answer'].apply(lambda x: len(x) if isinstance(x, list) else 1) < 3)]
    # Check if there are enough samples to sample from
    if len(filtered_df) >= n_samples:
        sampled_df = filtered_df.sample(n=n_samples, random_state=9999)
        processed_dfs.append(sampled_df)
    else:
        print(f"Not enough samples for Q_type {question}")

# Concatenate all processed dataframes
result_df = pd.concat(processed_dfs, ignore_index=True)

# Save the result to a new CSV file
print(len(result_df))
result_df.to_csv('Spar_balanced_120_1.csv', index=False)



364
Q_type Dictionary {'YN': 143, 'CO': 79, 'FR': 77, 'FB': 65}
140


In [101]:
# Load your dataset
df = pd.read_csv('spar_300_test.csv')
df1= df.head(130)

# Get the unique relations and their counts
relation_counts = df1['Q_type'].value_counts()
# Create a dictionary of relations
relation_dict = relation_counts.to_dict()
# Print the relation dictionary
print("Q_type Dictionary",relation_dict)



Q_type Dictionary {'FB': 38, 'YN': 32, 'CO': 31, 'FR': 29}


In [69]:
import pandas as pd

# Read the two CSV files
df2= pd.read_csv('5_ollama_models_Base.csv')
df1 = pd.read_csv('5_ollama_models_Base_full.csv')


# Concatenate the dataframes based on the 'Question' column
result = pd.concat([df1, df2], axis=0, ignore_index=True)

# Remove duplicate rows based on the 'Question' column, keeping the first occurrence
result = result.drop_duplicates(subset='Question', keep='first')

# Reset the index of the resulting dataframe
result = result.reset_index(drop=True)
relation_counts = result ['Q_type'].value_counts()
print(relation_counts)
# Save the result to a new CSV file
result.to_csv('spar_combined_1.csv', index=False)

print(f"Combined CSV file created with {len(result)} rows.")

df = pd.read_csv('spar_combined_1.csv')
counts = df['Q_type'].value_counts()
print(counts)

Q_type
FR    80
CO    79
YN    76
FB    57
Name: count, dtype: int64
Combined CSV file created with 292 rows.
Q_type
FR    80
CO    79
YN    76
FB    57
Name: count, dtype: int64


In [90]:
df = pd.read_csv('spar_300_test.csv')
df = df.sample(frac=1, random_state=42)
df.to_csv('spar_300_test_balanced.csv', index=False)


In [ ]:
# Load your dataset

import re
import pandas as pd
from typing import List, Union
from fuzzywuzzy import fuzz
import ast

df = pd.read_csv('Spar.csv')
print(len(df))
relations = df['target_relation'].unique()
# Get the unique relations and their counts
relation_counts = df['target_relation'].value_counts()
# Create a dictionary of relations
relation_dict = relation_counts.to_dict()
# Print the relation dictionary
print("Relation Dictionary",relation_dict)
relation_distribution = df['target_relation'].value_counts() 

# Define custom colors for the bars
custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

plt.figure(figsize=(12, 8))
relation_distribution.plot(kind='bar', color=custom_colors)
plt.title('Distribution of target_relation types in Spartun', fontsize=14, fontweight='bold')
plt.xlabel('Relation Types', fontsize=12)
plt.ylabel('Counts', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=11)
plt.grid(axis='y', linestyle='--', alpha=0.6)  # Add grid lines
plt.tight_layout()

plt.savefig('relation_distribution_plot.png', dpi=300)  # Save the plot with higher resolution
plt.show()

In [32]:
def calculate_accuracy(true_answers, pred_answers):
    correct = 0
    total = len(true_answers)

    def extract_answer(text):
        if isinstance(text, str):
            match = re.search(r'(answer|the answer is)\s*:\s*([a-zA-Z]+)', text, re.IGNORECASE)
            if match:
                return match.group(2).strip()
        return text  # Return the original text if no match is found or if it's not a string

    def normalize_answer(answer):
        if isinstance(answer, str):
            # First, try to extract the answer if it's in the format "answer: ..."
            answer = extract_answer(answer)
            # Then, try to evaluate if it's a string representation of a list
            try:
                lst = ast.literal_eval(answer)
               # Join list elements into a single string
                return  [','.join(str(item) for item in lst)]
            except:
                pass
        
        if isinstance(answer, list):
            return [str(item).lower().strip() for item in answer]
        else:
            return [str(answer).lower().strip()]

    for true, pred in zip(true_answers, pred_answers):
        if pd.isna(true) or pd.isna(pred):
            total -= 1  # Skip this pair if either is NaN
            continue

        true_set = set(normalize_answer(true))
        pred_set = set(normalize_answer(pred))

        if len(true_set) > 1:  # Multiple correct answers
            if true_set == pred_set:
                correct += 1  # Full match
            elif true_set.intersection(pred_set):
                correct += 0.4  # Partial match
        else:  # Single correct answer
            if true_set == pred_set:
                correct += 1

    return correct / total if total > 0 else 0

df_results = pd.read_csv('2_small_Base.csv')
models = ["ollama/phi3:mini","ollama/gemma:2b"]
metrics = {}
for model in models:
    accuracy = calculate_accuracy(df_results['Answer'], df_results[model])
    metrics[model] = {'accuracy': accuracy}
# Create a DataFrame with the metrics
metrics_df = pd.DataFrame(metrics).T
print("metrics for CoT prompt", metrics_df)

metrics for CoT prompt                   accuracy
ollama/phi3:mini  0.328000
ollama/gemma:2b   0.002667


In [99]:
def normalize_answer(answer):
        if isinstance(answer, str):
            # First, try to extract the answer if it's in the format "answer: ..."
    
            # Then, try to evaluate if it's a string representation of a list
            try:
                lst = ast.literal_eval(answer)
               # Join list elements into a single string
                return  [','.join(str(item) for item in lst)]
            except:
                pass
        
        if isinstance(answer, list):
            return [str(item).lower().strip() for item in answer]
        else:
            return [str(answer).lower().strip()]
re = set('apple')
print(re)

{'e', 'l', 'a', 'p'}


In [109]:
import pandas as pd
import ast
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_accuracy_transformer(true_answers: pd.Series, pred_answers: pd.Series) -> float:
    # Load a pre-trained sentence transformer model
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    def normalize(answer):
        if pd.isna(answer):
            return ""
        # Try to evaluate as a literal Python expression (for list-like strings)
        try:
            answer_list = ast.literal_eval(answer)
            if isinstance(answer_list, list):
                return ", ".join(str(item).strip() for item in answer_list)
        except:
            pass
        
        # If not a list, return as is
        return str(answer).strip()

    def compute_similarity(true, pred):
        # Encode sentences
        true_embedding = model.encode([true])
        pred_embedding = model.encode([pred])
        
        # Compute cosine similarity
        similarity = cosine_similarity(true_embedding, pred_embedding)[0][0]
        
        # Adjust similarity score to consider partial matches
        if similarity > 0.9:  # High similarity, consider as full match
            return 1.0
        elif similarity > 0.7:  # Moderate similarity, consider as partial match
            return 0.5
        else:
            return similarity  # Low similarity, return as is

    total_similarity = 0
    valid_comparisons = 0

    for true, pred in zip(true_answers, pred_answers):
        true_normalized = normalize(true)
        pred_normalized = normalize(pred)

        if not true_normalized or not pred_normalized:
            continue

        similarity = compute_similarity(true_normalized, pred_normalized)
        total_similarity += similarity
        valid_comparisons += 1

    accuracy = total_similarity / valid_comparisons if valid_comparisons > 0 else 0.0

    return accuracy


/opt/miniconda3/envs/autogen/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
